In [12]:
import geopandas as gpd
import streamlit as st
from streamlit_image_comparison import image_comparison

In [4]:
# load maps
access_score_adults = gpd.read_parquet('./results/access_score_adults.parquet')
access_score_seniors = gpd.read_parquet('./results/access_score_seniors.parquet')

In [20]:
st.set_page_config(page_title="Image-Comparison Example", layout="centered")

In [21]:
# image path

# image url
image = "https://tinypng.com/images/social/website.jpg"

# # pil image
# from PIL import Image
# image = Image.open("image.jpg")

# # opencv image
# import cv2
# image = cv2.cvtColor(cv2.imread("image.jpg"), cv2.COLOR_BGR2RGB)

# render image-comparison
image_comparison(
    img1=image,
    img2=image,
)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)